In [1]:
import os
import pandas as pd
import ipywidgets
import plotly.express as px
import glob

In [2]:

path = 'output/**/results.csv'
results = []
for file in glob.glob(path, recursive=True):
    results.append(file)
    
path = 'output/**/aispositions.csv'
aisresults = []
for file in glob.glob(path, recursive=True):
    aisresults.append(file)


In [3]:
aisds = []
for r in aisresults:
    aisds.append(pd.read_csv(r))


In [4]:
results

['output\\Helsingborg_2\\results.csv',
 'output\\march_1024\\results.csv',
 'output\\october_1024\\results.csv',
 'output\\Rotterdam_31_aug_22\\results.csv',
 'output\\Suez_05_Dec_2021\\results.csv']

In [9]:
rpds = []
for r in results:
    rpds.append(pd.read_csv(r))
rdf = pd.concat(rpds)
truec = rdf[rdf["4"]]
rdf["Class"] =rdf.apply(lambda row: "True" if 'october' in row[6] else "Unknown" if 'Suez' in row[6] else str(row["4"]), axis=1)
allais = rdf[rdf["Class"] != "Unknown"]

rdf['Set'] =rdf.apply(lambda row:  row[6][row[6].find('/')+7:row[6].find('_')], axis=1)
rdf.head()
rdf = rdf[rdf["length"]<500]

In [6]:
rdf.head()

,Unnamed: 0,0,1,2,3,4,5,6,length,Class,Set
0,0,503,806,9,12,False,./input/Helsinborg_23_aug_22_2/Helsinborg_23_a...,0,22.500000,False,Helsinborg
1,1,409,641,71,91,True,./input/Helsinborg_23_aug_22_2/Helsinborg_23_a...,4,173.131453,True,Helsinborg
2,2,635,649,17,13,False,./input/Helsinborg_23_aug_22_2/Helsinborg_23_a...,4,32.101402,False,Helsinborg
3,3,562,481,240,73,True,./input/Helsinborg_23_aug_22_2/Helsinborg_23_a...,4,376.284799,True,Helsinborg
4,4,697,554,300,142,True,./input/Helsinborg_23_aug_22_2/Helsinborg_23_a...,4,497.864439,True,Helsinborg


In [10]:
fig = px.histogram(rdf, x="length", color="Class", pattern_shape ="Set", title = f"Data: All Boats {len(rdf)} AIS: {100*len(allais) / len(rdf):.2f}%")
fig.show()


In [8]:
df["Class"].unique()

NameError: name 'df' is not defined

In [ ]:
import numpy as np
np.random.seed(10)

remove_n = 50
df = rdf[rdf["Class"].str.contains("False")]
drop_indices = np.random.choice(df.index, remove_n, replace=False)
df_subset = df.drop(drop_indices)

df_subset = pd.concat([df_subset, rdf[rdf["Class"]!="False"]])

In [ ]:
fig = px.histogram(df_subset, x="length", color="Class", pattern_shape ="Set", title = f"Data: All Boats {len(df_subset)} AIS: {100*len(allais) / len(rdf):.2f}%")
fig.show()


In [19]:
aisdf = pd.concat(aisds)
aisdf["sog"]

0      91.0
1      70.0
2      86.0
3       0.0
4       0.0
       ... 
135     0.0
136     0.0
137     0.0
138     0.0
0       1.0
Name: sog, Length: 225, dtype: float64

In [17]:

fig = px.scatter_geo(aisdf, lat="latitude", lon="longitude", color="sog", title="AIS ship location")
fig.show()

In [32]:
import plotly.graph_objects as go
justsinga = aisdf#[aisdf["longitude"]>100]
fig = go.Figure(px.scatter_geo(justsinga, lat="latitude", lon="longitude", color="sog", title="AIS ship location"))
#fig.update_geos(
#    resolution=50,
#    showcoastlines=True, coastlinecolor="RebeccaPurple",
#    showland=True, landcolor="LightGreen",
#    showocean=True, oceancolor="LightBlue",
#    showlakes=True, lakecolor="Blue",
#    showrivers=True, rivercolor="Blue"
#)
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [28]:
len(aisdf)

225

In [29]:
aisdf.to_csv("allAIS-data.csv")
rdf.to_csv("all_detected_ships.csv")

In [33]:
fig = px.scatter_mapbox(aisdf, lat="latitude", lon="longitude", hover_name="mmsi",
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()